# Solutions to homework week 3 - logistic regression

In [2]:
import pandas as pd
import numpy as np

In [3]:
# check working directory to debug path to loading
#import os
#os.getcwd()

# load the data
df = pd.read_csv('../data/lead_scoring.csv')

In [10]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [ ]:
# df.lead_source.value_counts().head()
categorical_columns = ['lead_source', 'industry', 'employment_status', 'location']
numeric_columns = ['number_of_courses_viewed', 'annual_income','interaction_count','lead_score']

### Data preparation

* Check if the missing values are presented in the features.
* If there are missing values:
    * For caterogiral features, replace them with 'NA'
    * For numerical features, replace with with 0.0 

In [46]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [ ]:
missing_values_dict = {}
for c in categorical_columns:
    missing_values_dict[c] = 'NA'

for n in numeric_columns:    
    missing_values_dict[n] = 0
# df.fillna(value=values)
df = df.fillna(value=missing_values_dict)

In [48]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

### Question 1

What is the most frequent observation (mode) for the column `industry`?


In [49]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

### Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset. 
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

In [60]:
df[numeric_columns].corr().round(3)


,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000,0.010,-0.024,-0.005
annual_income,0.010,1.000,0.027,0.016
interaction_count,-0.024,0.027,1.000,0.010
lead_score,-0.005,0.016,0.010,1.000


### Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
* Make sure that the target value `y` is not in your dataframe.


In [61]:
from sklearn.model_selection import train_test_split

In [ ]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
# adjust proportion test size to be 20% of total, after reduction by 20%
df_train, df_val = train_test_split(df_full_train, test_size=0.2/0.8, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df_full_train = df_full_train.reset_index(drop=True)

y_full_train = df_full_train.converted.values
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_full_train['converted']
del df_train['converted']
del df_val['converted']
del df_test['converted']


### Question 3

* Calculate the mutual information score between `y` and other categorical variables in the dataset. Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

In [65]:
from sklearn.metrics import mutual_info_score

In [67]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, y_train)

mi = df_train[categorical_columns].apply(mutual_info_churn_score)
mi.sort_values(ascending=False).round(2)

lead_source          0.03
employment_status    0.02
industry             0.01
location             0.00
dtype: float64

### Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [68]:
from sklearn.feature_extraction import DictVectorizer

In [70]:
# instantiate class vectorizer
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numeric_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numeric_columns].to_dict(orient='records')
# validation uses the one-hot-encoding fit from training
X_val = dv.transform(val_dict)

In [71]:
from sklearn.linear_model import LogisticRegression

In [108]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# probility of event is second column
y_pred = model.predict_proba(X_val)[:,1]

# proportion of records correctly identified
accuracy_all_features = (y_val == (y_pred >= 0.5)).mean()
accuracy_all_features.round(2)

np.float64(0.74)

### Question 5 

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model using the same features and parameters as in Q4 (without rounding).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 

Which of following feature has the smallest difference?

In [ ]:
def model_fit(features):
    # instantiate class vectorizer
    dv = DictVectorizer(sparse=False)

    train_dict = df_train[features].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val[features].to_dict(orient='records')
    # validation uses the one-hot-encoding fit from training
    X_val = dv.transform(val_dict)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    # probility of event is second column
    y_pred = model.predict_proba(X_val)[:,1]

    # proportion of records correctly identified
    return((y_val == (y_pred >= 0.5)).mean())

def model_feature_loop(features):
    accuracy_dict = {}
    accuracy_list = []

    for feature_dropped in features:
        feature_subset = features.copy()
        feature_subset.remove(feature_dropped)
        
        accuracy_dict[feature_dropped] = model_fit(feature_subset)
        accuracy_list.append(model_fit(feature_subset))
        
    return(accuracy_dict)

accuracy_dict = model_feature_loop(categorical_columns + numeric_columns)

accuracy_series = pd.Series(accuracy_dict)

accuracy_abs_diff = abs(accuracy_all_features - accuracy_series)

# question interested in differenct in only subset of features
accuracy_abs_diff.loc[['industry', 'employment_status', 'lead_score']].sort_values(ascending=True)


lead_score           0.003413
industry             0.010239
employment_status    0.051195
dtype: float64

### Question 6

* Now let's train a regularized logistic regression.
* Let's try the following values of the parameter `C`: `[0.01, 0.1, 1, 10, 100]`.
* Train models using all the features as in Q4.
* Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

In [153]:
accuracy_list = []
r_parameters = [0.01, 0.1, 1, 10, 100]

features = categorical_columns + numeric_columns

for r in r_parameters:
    # instantiate class vectorizer
    dv = DictVectorizer(sparse=False)

    train_dict = df_train[features].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val[features].to_dict(orient='records')
    # validation uses the one-hot-encoding fit from training
    X_val = dv.transform(val_dict)

    model = LogisticRegression(solver='liblinear', C=r, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    # probility of event is second column
    y_pred = model.predict_proba(X_val)[:,1]

    # proportion of records correctly identified
    accuracy_r = (y_val == (y_pred >= 0.5)).mean()

    accuracy_list.append(accuracy_r.round(3))

pd.DataFrame(data = accuracy_list, index = r_parameters, columns = ['accuracy']).sort_values('accuracy')

,accuracy
0.01,0.744
0.10,0.744
1.00,0.744
10.00,0.744
100.00,0.744
